In [1]:
# import libraries
import os
import numpy as np
import pandas as pd

In [2]:
# change working path
os.chdir('../')
os.getcwd()

'C:\\Users\\MDelosSanto\\space\\dev\\ptsa'

In [3]:
# load year_joined table
year_joined = pd.read_csv('data/input/nonprofit_org/year_joined.csv')

In [4]:
year_joined.head()

,year_joined,user_status
0,2018,inactive
1,2018,silver
2,2014,bronze
3,2017,silver
4,2015,silver


In [5]:
(
        year_joined.groupby('year_joined').count().reset_index()
)

,year_joined,user_status
0,2014,90
1,2015,97
2,2016,169
3,2017,295
4,2018,349


In [6]:
(
        year_joined.groupby(['year_joined', 'user_status']).size()
)

year_joined  user_status
2014         bronze          45
             gold             9
             inactive        13
             silver          23
2015         bronze          49
             gold             9
             inactive        14
             silver          25
2016         bronze          99
             gold            12
             inactive        12
             silver          46
2017         bronze         167
             gold            31
             inactive        17
             silver          80
2018         bronze         176
             gold            25
             inactive        44
             silver         104
dtype: int64

In [7]:
(
        year_joined.groupby(['year_joined', 'user_status']).size().reset_index().head()
)

,year_joined,user_status,0
0,2014,bronze,45
1,2014,gold,9
2,2014,inactive,13
3,2014,silver,23
4,2015,bronze,49


In [8]:
(
    year_joined.groupby('year_joined').count().
                groupby('user_status').count().
                reset_index().
                sum()
)

user_status    1000
dtype: int64

In [9]:
# load emails table
emails = pd.read_csv('data/input/nonprofit_org/emails.csv')

In [10]:
emails.head()

,user,week,emails_opened
0,2.0,2015-02-09 00:00:00,3.0
1,2.0,2015-02-16 00:00:00,3.0
2,2.0,2015-02-23 00:00:00,3.0
3,2.0,2015-03-02 00:00:00,3.0
4,2.0,2015-03-09 00:00:00,3.0


In [11]:
emails[emails['emails_opened'] < 1]

,user,week,emails_opened


In [12]:
# check history of one user
emails[emails['user'] == 998]

,user,week,emails_opened
28535,998.0,2015-02-09 00:00:00,2.0
28536,998.0,2015-02-16 00:00:00,2.0
28537,998.0,2015-02-23 00:00:00,2.0
28538,998.0,2015-03-16 00:00:00,1.0
28539,998.0,2015-03-23 00:00:00,2.0
28540,998.0,2015-03-30 00:00:00,2.0
28541,998.0,2015-04-06 00:00:00,1.0
28542,998.0,2015-05-04 00:00:00,1.0
28543,998.0,2015-06-22 00:00:00,1.0
28544,998.0,2015-08-03 00:00:00,1.0


In [13]:
# we can observe there are weeks missing

We  can  check  this  more  mathematically  by  calculating  how  many  weekly  observa‐tions we should have between the first and last event for that member. First we calcu‐late the length of the member’s tenure, in weeks:

In [14]:
emails['week'] = pd.to_datetime(emails['week'])

In [15]:
(
    max(emails[emails['user'] == 998]['week']) - min(emails[emails['user'] == 998]['week'])
).days / 7

25.0

Then we see how many weeks of data we have for that member:

In [16]:
emails[emails['user'] == 998].shape

(10, 3)

We have 10 rows here, but we should have 26. This  shows  some  weeks  of  data  aremissing for this member.

### Why 26 Rows?
You may be surprised that we need 26 instead of 25 given the subtraction we just per‐formed,  but  that  was  an  incomplete  calculation.  When  you  work  with  time  seriesdata, one thing you should always ask yourself after doing this kind of subtraction iswhether you should add 1 to account for the offset at the end. In other words, did yousubtract the positions you wanted to count?

Consider  this  example.  Let’s  say  I  have  information  for  April  7th,  14th,  21st,  and28th. I want to know how many data points I should have in total. Subtracting 7 from28  and  dividing  by  7  yields  21/7  or  3.  However,  I  should  obviously  have  four  datapoints. I subtracted out April 7th and need to put it back in, so the proper calculationis the difference between the first and last days divided by 7, plus 1 to account for thesubtracted start date.

We’ll move on to filling in the blanks so that we have a complete data set now that wehave confirmed we do indeed have missing weeks. We can’t be sure of identifying allmissing  weeks,  since  some  may  have  occurred  before  our  earliest  recorded  date  orafter  our  last  recorded  date.  What  we  can  do,  however,  is  fill  in  the  missing  valuesbetween the first and last time a member had a non-null event.

It’s  a  lot  easier  to  fill  in  all  missing  weeks  for  all  members  by  exploiting  Pandas’indexing  functionality,  rather  than  writing  our  own  solution.  We  can  generate  aMultiIndex for a Pandas data frame, which will create all combinations of weeks andmembers—that is, a Cartesian product:

In [17]:
complete_idx = pd.MultiIndex.from_product((set(emails['week']), set(emails['user'])))

In [18]:
complete_idx

MultiIndex([('2018-03-26',   2.0),
            ('2018-03-26',   3.0),
            ('2018-03-26',   4.0),
            ('2018-03-26',   5.0),
            ('2018-03-26',   9.0),
            ('2018-03-26',  12.0),
            ('2018-03-26',  14.0),
            ('2018-03-26',  15.0),
            ('2018-03-26',  17.0),
            ('2018-03-26',  18.0),
            ...
            ('2015-10-12', 987.0),
            ('2015-10-12', 988.0),
            ('2015-10-12', 989.0),
            ('2015-10-12', 990.0),
            ('2015-10-12', 994.0),
            ('2015-10-12', 995.0),
            ('2015-10-12', 996.0),
            ('2015-10-12', 997.0),
            ('2015-10-12', 998.0),
            ('2015-10-12', 999.0)],
           length=99648)

In [19]:
len(complete_idx)

99648

We use this index to reindex the original table and fill in the missing values—in thiscase  with  0  on  the  assumption  that  nothing  recorded  means  there  was  nothing  torecord. We also reset the index to make the member and week information availableas columns, and then name those columns:

In [20]:
all_email = (
    emails.set_index(['week', 'user']).
    reindex(complete_idx, fill_value = 0).
    reset_index()
)

In [21]:
all_email.columns = ['week', 'user', 'emails_opened']

Let’s take a look at member 998 again:

In [22]:
all_email[all_email['user'] == 998].sort_values('week')

,week,user,emails_opened
74302,2015-02-09,998.0,2.0
89854,2015-02-16,998.0,2.0
2302,2015-02-23,998.0,2.0
40894,2015-03-02,998.0,0.0
30526,2015-03-09,998.0,0.0
...,...,...,...
33982,2018-04-30,998.0,0.0
51838,2018-05-07,998.0,0.0
79486,2018-05-14,998.0,0.0
55294,2018-05-21,998.0,0.0


Notice  that  we  have  a  large  number  of  zeros  at  the  start.  These  are  likely  before  themember joined the organization, so they would not have been on an email list. Thereare not too many kinds of analyses where we’d want to keep the member’s truly nullweeks  around—specifically  those  weeks  before  the  member  ever  indicated  openingan  email.  If  we  had  the  precise  date  a  member  started  receiving  emails,  we  wouldhave  an  objective  cutoff.  As  it  is,  we  will  let  the  data  guide  us.  For  each  member  wedetermine  the  start_date  and  end_date  cutoffs  by  grouping  the  email  DataFrameper member and selecting the maximum and minimum week values:

In [23]:
cutoff_dates = (
    emails.groupby('user').week.
    agg(['min', 'max']).
    reset_index()
)

In [24]:
cutoff_dates = cutoff_dates.reset_index()

We drop rows from the DataFrame that don’t contribute sensibly to the chronology,specifically 0 rows before each member’s first nonzero count:

In [25]:
for __, row in cutoff_dates.iterrows():
    # define user and its info
    user = row['user']
    start_date = row['min']
    end_date = row['max']
    # drop 0s before start_date
    all_email.drop(
        all_email[(all_email['user'] == user) &
                  (all_email['week'] < start_date)].index,
        inplace = True
    )
    # drop 0s after end_date
    all_email.drop(
        all_email[(all_email['user'] == user) &
                  (all_email['week'] > end_date)].index,
        inplace = True
    )

In [26]:
all_email[all_email['user'] == 998].sort_values(by = 'week')

,week,user,emails_opened
74302,2015-02-09,998.0,2.0
89854,2015-02-16,998.0,2.0
2302,2015-02-23,998.0,2.0
40894,2015-03-02,998.0,0.0
30526,2015-03-09,998.0,0.0
59902,2015-03-16,998.0,1.0
37438,2015-03-23,998.0,2.0
20734,2015-03-30,998.0,2.0
32254,2015-04-06,998.0,1.0
44350,2015-04-13,998.0,0.0


### Constructing a Found Time Series

Consider how to relate the email and donations data to one another. We can down‐sample the donation data to turn it into a weekly time series, comparable to the emaildata. As an organization, we are interested in the total weekly amounts, so we aggre‐gate  the  timestamps  into  weekly  periods  by  summing.  More  than  one  donation  in  aweek is unlikely, so the weekly donation amounts will reflect the individual donationamounts for most donors

In [27]:
donations = pd.read_csv('data/input/nonprofit_org/donations.csv')

In [28]:
donations.head()

,user,amount,timestamp
0,2.0,25.0,2015-02-19 14:22:17
1,2.0,50.0,2015-08-14 14:12:05
2,2.0,25.0,2017-09-09 14:25:50
3,2.0,50.0,2017-09-14 14:06:47
4,2.0,25.0,2017-07-31 18:41:25


In [29]:
donations['timestamp'] = pd.to_datetime(donations['timestamp'])

In [30]:
donations.set_index('timestamp', inplace = True)

In [31]:
agg_donations = (
    donations.groupby('user').
    apply(
        lambda df: df['amount'].resample('W-MON').
        sum().
        dropna()
    )
)

In [32]:
agg_donations = agg_donations.reset_index()

In this code we first convert a string character to a proper timestamped data class soas to benefit from Pandas’ built-in date-related indexing. We set the timestamp as anindex, as is necessary for resampling a data frame. Finally for the data frame that weobtain from subsetting down to each member, we group and sum donations by week,drop weeks that do not have donations, and then collect these together.

Note  that  we  resampled  with  an  anchored  week  so  that  we  will  match  the  sameweekly  dates  we  already  have  in  our  email  table.  Note  also  that  a  week  anchored  to“Monday” makes sense from a human perspective.

We now have donation information and email information sampled at the same fre‐quency,  and  we  can  join  them.  Pandas  makes  this  simple  so  long  as  we  anchor  theweeks  to  the  same  day  of  the  week,  as  we’ve  done  already.  We  can  iterate  througheach member and merge the data frames per member:

In [34]:
merged_df = pd.DataFrame()
for user, user_email in all_email.groupby('user'):
    user_donations = agg_donations[agg_donations['user'] == user]
    user_donations.set_index('timestamp', inplace = True)
    user_email.set_index('week', inplace = True)
    user_email = all_email[all_email['user'] == user]
    user_email.sort_values('week').set_index('week')
    df = pd.merge(user_email, user_donations, how = 'left',
                  left_index = True,
                  right_index = True)
    df.fillna(0)
    df['user'] = df['user_x']
    merged_df = merged_df.append(
        df.reset_index()[['user', 'week', 'emails_opened', 'amount']]
    )

In [35]:
merged_df

,user,week,emails_opened,amount
0,2.0,2018-03-26,3.0,NaN
1,2.0,2017-10-30,3.0,NaN
2,2.0,2017-01-23,3.0,NaN
3,2.0,2015-02-23,3.0,NaN
4,2.0,2016-04-04,3.0,NaN
...,...,...,...,...
99,999.0,2017-03-27,2.0,NaN
100,999.0,2017-10-02,1.0,NaN
101,999.0,2018-01-08,0.0,NaN
102,999.0,2017-04-17,1.0,NaN


We now have our email and donations data lined up per member. For each memberwe  include  only  meaningful  weeks,  and  not  weeks  that  appear  to  be  before  or  aftertheir membership period.